# Inhalt

    k_import
    k_peaks
    k_mittelwerte
    k_maxwerte
    k_filter
    k_start_ende
    k_detrend

In [6]:
import pickle
import numpy as np
import os,glob
import scipy.io.wavfile
import scipy.signal
import timeit
from datetime import datetime, timedelta
from scipy.signal import find_peaks

def k_import(trennzeichen="tab", speicherort=''):

    if trennzeichen == 'tab':
        trennzeichen = '\t'
    
    Ordner = speicherort
    
    Messungen = 0

    for filename in glob.glob(os.path.join(Ordner, '*.txt')):
        Messungen = Messungen + 1

    print("Opening file: " + filename[len(Ordner):-4] + " to calculate length")

    data = ""
    with open(filename, 'r') as file:
        data = file.read().replace(',', '.')

    with open(filename, "w") as out_file:
        out_file.write(data)

    length = len(np.loadtxt(filename, delimiter=trennzeichen, skiprows=3, dtype=float))

    Zeit = np.zeros((length,Messungen))
    Fx = np.zeros((length,Messungen))
    Fy = np.zeros((length,Messungen))
    Fz = np.zeros((length,Messungen))

    x = 1

    print('Beginning file import...') 

    start_time = timeit.default_timer()
    
    for filename in glob.glob(os.path.join(Ordner, '*.txt')):

        print("File " + str(x) + ": " +  filename[len(Ordner):-4], end =" ")

        data = ""
        with open(filename, 'r') as file:
            data = file.read().replace(',', '.')

        with open(filename, "w") as out_file:
            out_file.write(data)

        Messdaten = np.loadtxt(filename, delimiter=trennzeichen, skiprows=3, dtype=float)

        Zeit[:,(x-1)] = Messdaten[:,0]
        Fx[:,(x-1)] = Messdaten[:,1]
        Fy[:,(x-1)] = Messdaten[:,2]
        Fz[:,(x-1)] = Messdaten[:,3]

        print(' - succesfull', end =" ") 
    
        stop_time = timeit.default_timer()
        laufzeit = np.round(((stop_time-start_time)/x * (Messungen-x)))

        sometime = datetime.now()
        fertig = sometime + timedelta(seconds=laufzeit)

        print(' - geschaetzte Fertigstellung:', fertig.strftime('%H:%M'))

        x = x + 1

    print('Import complete') 


    with open('Kraftsignale.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump([Zeit, Fx, Fy, Fz], f)

def k_peaks(Zeit, messdaten, start, ende, distance):            
    
    abtastrate = int(np.round(1/(Zeit[1,0]-Zeit[0,0])))
    
    messdaten = np.asarray(messdaten)
    [a, b, c] = np.shape(messdaten)
    peaks = np.empty_like(messdaten)
    #peaks[:] = np.nan
    
    for i in range(0,a):
        for k in range(0, c):
            if isinstance(start, int):
                peaks_array, _ = find_peaks(messdaten[i, int(start*abtastrate):int(ende*abtastrate), k],
                                        distance=distance)
                peaks_array = peaks_array + int(start*abtastrate)
                peaks[i, peaks_array, k] = messdaten[i, peaks_array, k]
            else:
                peaks_array, _ = find_peaks(messdaten[i, int(start[k]*abtastrate):int(ende[k]*abtastrate), k],
                                        distance=distance)
                peaks_array = peaks_array + int(start[k]*abtastrate)
                peaks[i, peaks_array, k] = messdaten[i, peaks_array, k]
    
    peaks[peaks == 0] = np.nan
    
    return peaks

def k_mittelwerte(Zeit, messdaten, start, ende):
    
    abtastrate = int(np.round(1/(Zeit[1,0]-Zeit[0,0])))
    
    messdaten = np.asarray(messdaten)
    [a, b, c] = np.shape(messdaten)
    mittelwert = np.zeros([a, c])
    
    for i in range(0,a):
        for k in range(0, c):
            if isinstance(start, int):
                if ende > b:
                    raise ValueError('Das gewaehlte Zeitfenster geht ueber die Messdauer hinaus, bitte kuerzen (ende > Messdauer)')
                mittelwert[i, k] = np.nanmean(messdaten[i, int(start*abtastrate):int(ende* btastrate), k])
            else:
                if ende[k] > b:
                    raise ValueError('Das gewaehlte Zeitfenster geht ueber die Messdauer hinaus, bitte kuerzen (ende > Messdauer)')
                mittelwert[i, k] = np.nanmean(messdaten[i, int(start[k]*abtastrate):int(ende[k]*abtastrate), k])

    return mittelwert

def k_maxwerte(Zeit, messdaten, start, ende):
    
    abtastrate = int(np.round(1/(Zeit[1,0]-Zeit[0,0])))
    
    messdaten = np.asarray(messdaten)
    [a, b, c] = np.shape(messdaten)
    mittelwert = np.zeros([a, c])
    
    for i in range(0,a):
        for k in range(0, c):
            if isinstance(start, int):
                if ende > b:
                    raise ValueError('Das gewaehlte Zeitfenster geht ueber die Messdauer hinaus, bitte kuerzen (ende > Messdauer)')
                mittelwert[i, k] = np.nanmax(messdaten[i, int(start*abtastrate):int(ende* btastrate), k])
            else:
                if ende[k] > b:
                    raise ValueError('Das gewaehlte Zeitfenster geht ueber die Messdauer hinaus, bitte kuerzen (ende > Messdauer)')
                mittelwert[i, k] = np.nanmax(messdaten[i, int(start[k]*abtastrate):int(ende[k]*abtastrate), k])

    return mittelwert

def k_filter(Zeit, messdaten, filterwert):
    
    abtastrate = int(np.round(1/(Zeit[1,0]-Zeit[0,0])))
    filterfrequenz = 1/abtastrate * filterwert

    messdaten = np.asarray(messdaten)
    gefiltert = np.empty_like(messdaten)
    [a, b, c] = np.shape(messdaten)
        
    order = 4
    d, e = scipy.signal.butter(order, filterfrequenz)

    for i in range(0,a):
        for k in range(0, c):
            gefiltert[i,:,k] = scipy.signal.filtfilt(d, e, messdaten[i,:,k])
            
    return gefiltert

# Start und Ende finden

def k_start_ende(Zeit, Fy_f, anstiegsdauer_1=0.05, anstiegsdauer_2=0.5, min_messzeit = 0.1, not_start=2, not_ende=4, plot=False):
    
    import matplotlib.pyplot as plt
    
    abtastrate = int(np.round(1/(Zeit[1,0]-Zeit[0,0])))
    anstiegsdauer_1 = int(abtastrate*anstiegsdauer_1)  
    anstiegsdauer_2 = int(abtastrate*anstiegsdauer_2)
    
    # Sprung berechnen (grobe Steigung)
    sprung = np.empty_like(Fy_f)
    [a,b] = Zeit.shape
    
    for w in range(0, b):
        k = a - abtastrate  
        for i in range(0, k):
            k = i + anstiegsdauer_1
            sprung[i,w] = Fy_f[i,w] - Fy_f[k,w]
            
    # findet die Stelle mit dem größten Antieg und Abfall
    start = np.empty(b)
    ende = np.empty(b)
    
    for w in range(0, b): 
        min_pos = np.where(sprung[:,w] == np.amin(sprung[:,w]))
        max_pos = np.where(sprung[:,w] == np.amax(sprung[:,w]))
        start[w] = int(min_pos[0])
        ende[w] = int(max_pos[0])
        
    # findet das untere Niveau bei start und ende
    
    start_such_array = np.empty_like(Fy_f)
    end_such_array = np.empty_like(Fy_f)

    for w in range(0, b):
        
        # Sonderfälle falls der berechnete Startpunkt/Endpunkt zu nah am Anfang/Ende sind 
        # und die Berechnung daher nicht möglich ist
        if (start[w] - anstiegsdauer_2/2) < 0:
            continue
        if (ende[w] - anstiegsdauer_2/2) < 0:
            continue
        if (ende[w] + anstiegsdauer_2/2) > len(Zeit):
            continue

        start_such_array = Fy_f[int(start[w]-anstiegsdauer_2/2):int(start[w]+anstiegsdauer_2/2),w]
        end_such_array = Fy_f[int(ende[w]-anstiegsdauer_2/2):int(ende[w]+anstiegsdauer_2/2),w]  

        i = np.where(start_such_array == np.amin(start_such_array))
        k = np.where(end_such_array == np.amin(end_such_array))

        start[w] = int(i[0]+int(start[w]-anstiegsdauer_2/2))
        ende[w] = int(k[0]+int(ende[w]-anstiegsdauer_2/2))
        
        if plot:
            
            #Für alle möglichen Optionen
            #plt.rcParams.keys() 

            # Diagrammgröße
            plt.rcParams["figure.figsize"] = (10/2.54,6/2.54) # /2.54 inch to cm 

            # Linien und Marker
            plt.rcParams['lines.linestyle'] = '-'
            plt.rcParams["lines.linewidth"] = 0
            plt.rcParams["lines.markersize"] = .5
            plt.rcParams["lines.marker"] = 'o'
            plt.rcParams["lines.marker"] = 'o'

            # Tics
            plt.rcParams["axes.grid"] = True
            plt.rcParams['axes.axisbelow'] = True
            plt.rcParams["xtick.minor.visible"] = True

            # Schriftgröße
            plt.rcParams['font.size'] = 8
            plt.rcParams['axes.titlesize'] = 8
            plt.rcParams['axes.labelsize'] = 8
            plt.rcParams['xtick.labelsize'] = 8
            plt.rcParams['ytick.labelsize'] = 8
            plt.rcParams['legend.fontsize']  = 8

            # Legende
            plt.rcParams["legend.loc"] =  'upper right' # lower upper left right center best

            # Ausgabeart
            #%matplotlib inline  
            plt.rcParams['figure.dpi'] = 125 # Regelt auch die Größe des Diagramms in der Notebook Darstellung
            plt.rcParams["figure.facecolor"] = "white"

            bis = Zeit.shape[1]

            for Messbahn in range(0,bis):

                fig, ax = plt.subplots() #start des Plots
                x = Zeit[:,Messbahn]
                y = [Fy_f[:,Messbahn]] # [] muss auch genutzt werden wenn nur ein array geplottet werden soll
                label = ['Kraft - gefiltert'] # hier auch

                for i in range(0,len(y)):
                    pos = ax.plot(x, y[i], label = label[i])

                x = Zeit[int(start[Messbahn]), Messbahn]
                y = 0
                ax.plot(x, y, ms=5, label='start')

                x = Zeit[int(ende[Messbahn]), Messbahn]
                y = 0
                ax.plot(x, y, ms=5, label='ende')

                #ax.set_title(str(Messbahn) + '. Messbahn')
                ax.set_xlabel('Zeit in $s$')
                ax.set_ylabel('Kraft in $N$')

                # Legende aufhübschen
                legend = ax.legend(loc= 'best') #(framealpha = 0.5, bbox_to_anchor=(1.33,1.02)) 
                #for legend_handle in legend.legendHandles:
                #    legend_handle._legmarker.set_markersize(2)

                #plt.show()

                # Diagramm als .png speichern
                fig.savefig(('Kontrolle_start_ende_' + str(Messbahn)), bbox_inches='tight')
                plt.close()
            
    
    # Falls keine sinvollen start und endpunkte gefunden wurden

    problem_kraftmessung = []
    
    for w in range(0,b):
        if start[w] >= ende[w] or (ende[w]-start[w] < min_messzeit*abtastrate):
            start[w] = not_start*abtastrate
            ende[w] = not_ende*abtastrate
            problem_kraftmessung.append(w)
            
    return start, ende, problem_kraftmessung, sprung

def k_detrend(Fy_f, start, ende):
    
    [a,b] = Fy_f.shape

    Fy_k = np.copy(Fy_f)

    for w in range(0,b):
        korrektur = (Fy_f[int(start[w]),w]-Fy_f[int(ende[w]),w])/(ende[w]-start[w])
        for i in range(int(start[1]), a):
            Fy_k[i,w] = Fy_f[i,w] +  korrektur * (i - start[w])
            
    # Nullpunkt korrigieren
    
    for w in range(0,b):
        Fy_k[:,w] = Fy_k[:,w] - Fy_k[int(start[w]),w]
        
    return Fy_k